<div style="font-size:22pt; line-height:25pt; font-weight:bold; text-align:center;">Regularization of Neural Networks with  Dropout and DropConnect</div>

# <a id="sec0">0. Preparation</a>

In this notebook, we'll be using `torch` and `torchvision`. Please refer to the [PyTorch](https://pytorch.org/get-started/locally/) website for installation instructions. 
We will also be using `pandas`,`numpy`,`matplotlib` and `sklearn`.

In [1]:
import torch
import torchvision
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

# 1. <a id="sec1"></a> Introduction and motivation

Neural network (NN) models are very useful when it comes to domains where are used large labeled datasets. It is linked to the fact that their capacity can easily be increased by adding more layers or more units in each layer.
But when it comes to big networks, using millions or even billions of parameters can easily overfit even the largest of datasets.

We know a few forms of regularization, for instance Bayesian methods (Mackay, 1995), weight elimination (Weigend et al., 1991), and early stopping of training.

But recently, Hinton et al. created a new form of regularization called **Dropout** (Hinton et al., 2012). For each training example, forward propagation involves randomly deleting half the activations in each layer. The error is then backpropagated only through the remaining activations. Extensive experiments show that this significantly reduces over-fitting and improves test performance. 


In the paper [Regularization of Neural Networks using DropConnect](http://proceedings.mlr.press/v28/wan13.pdf), the authors propose DropConnect which generalizes Dropout by randomly dropping the weights rather than the activations. Like Dropout, the technique is suitable for fully connected layers only. 

This notebook aims at introducing Dropout, its implementation, and to compare it to No-Drop networks on the Fashion MNIST, and the MNIST. It also aims at introducing DropConnect, the model used by the authors of [Regularization of Neural Networks using DropConnect](http://proceedings.mlr.press/v28/wan13.pdf), the results they found, and propositions of implementations. 


1. [Introduction and motivation](#sec1)
2. [Theroy and implementation of DropOut](#sec2)
2. [Comparing Dropout to No-Drop : Fashion MNIST example](#sec3)
3. [Comparing Dropout to No-Drop : the MNIST example](#sec4)
3. [A glance at DropConnect theory](#sec5)
4. [DropConnect propositions of implementation](#sec6)

# 2. <a id="sec2"></a> Theory and Implementation of DropOut 

DropOut is a way to prevent overfitting. The key idea is to randomly drop units (along with their connections) from the neural network during training. 

<img src="img/dropout_network.png">
Srivastava, Nitish, et al. "Dropout: a simple way to prevent neural networks from overfitting." The journal of machine learning research 15.1 (2014): 1929-1958.



Dropout was proposed by (Hinton et al., 2012) as a form of regularization for fully connected neural network layers. Each element of a layer’s output is kept with probability *p*, otherwise being set to 0 with probability *(1 − p)*. Extensive experiments show that Dropout improves the network’s generalization ability, giving improved test performance.

When Dropout is applied to the outputs of a fully connected layer, we can write :

$$r=m*a(W\upsilon)$$
where :
- $r = [r_1,r_2,r_3,\dots,r_d]^T$ output of the layer
- $m$ is a binary mask vector of size d with each element, $j$, drawn independently from $m_j$ ∼ $Bernoulli(p)$
- $*$ denotes [element wise product](https://en.wikipedia.org/wiki/Hadamard_product_(matrices)#:~:text=In%20mathematics%2C%20the%20Hadamard%20product,of%20the%20original%20two%20matrices.).
- $W$ weight parameters (of size $d$ × $n$)
- $\upsilon = [\upsilon_1,\upsilon_2,\upsilon_3,\dots,\upsilon_n]^T$ input of the layer 

Biases are included in W with a corresponding fixed input of 1 for simplicity.






For common activation functions such as $tanh$, $centered$ $sigmoid$ and $reLU$ this equation can be re-written as:
$$r=a(m*W\upsilon)$$

where Dropout is applied at the inputs to the activation function.

<div class="alert alert-warning">
    
**Exercice** <br>
Can you prove the precedent assertion for $tanh$ and $reLU$ ?</div>

**Reminder of these two activation functions**

- hyperbolic tangent
$$a(x) = \frac{e^{x} - e^{-x}}{e^{x} + e^{-x}}$$


- Rectified Linear Unit (ReLU)
$$a(x) = \max\{0,x\}$$

<div class="alert alert-success">
    
**Help** 
We have $a(0)=0$ for both these functions.
</div>



Torch uses spatial dropout, which is described here: https://arxiv.org/pdf/1411.4280.pdf <br>
The function to use is:

`torch.nn.Dropout(p: float = 0.5, inplace: bool = False)`

In [2]:
from torch import nn

<div class="alert alert-warning">
    
**Exercice** <br>
Use torch to create a network that includes at least one Dropout layer. For instance you can create the different layers :
- (780,500)
- Dropout layer with probability 0.25
- (500,10)
</div>

In [3]:
# %load solutions/ex1.py

# 3. <a id="sec3"></a> Comparing Dropout to No-Drop : Fashion MNIST example



[Fashion-MNIST](https://github.com/zalandoresearch/fashion-mnist) is a dataset of Zalando's article images—consisting of a training set of 60,000 examples and a test set of 10,000 examples. Each example is a 28x28 grayscale image, associated with a label from 10 classes. Fashion-MNIST is a direct drop-in replacement for the original MNIST dataset for benchmarking machine learning algorithms. It shares the same image size and structure of training and testing splits but is more complex.

In [4]:
labels_text = ["T-shirt/top", "Trouser", "Pullover", "Dress", "Coat", "Sandal", "Shirt", "Sneaker", "Bag", "Ankle boot"]

PyTorch comes with this dataset by default, but we need to download it. We'll then make dataloaders which lazily iterate through the datasets. Just as we did the Deep Learning notebook, we'll use a training set and a validation set and greatly reduce their sizes, to have a first idea of the utility of Dropout in a reasonable time.

In [5]:
import torchvision.transforms as transforms

transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize((0.5,), (0.5,))])

full_trainset = torchvision.datasets.FashionMNIST(root='../data', train=True, download=True, transform=transform)
trainset, full_validset = torch.utils.data.random_split(full_trainset, (10000, 50000))
validset, _ = torch.utils.data.random_split(full_validset, (1000, 49000))

trainloader = torch.utils.data.DataLoader(trainset, batch_size=4, shuffle=True, num_workers=2)
validloader = torch.utils.data.DataLoader(validset, batch_size=4, shuffle=False, num_workers=2)

In [6]:
testset = torchvision.datasets.FashionMNIST(root='../data', train=False, download=True, transform=transform)

<div class="alert alert-warning">
    
**Exercice** <br>
1. Can you create a SimpleNet, using torch and nn.Linear, knowing that our data is in 28 to 28, and that we would want it to enter the layer as a 784 to 1 ?
2. Can you complete the training function ?
</div>

In [44]:
# %load solutions/ex2_1.py

We are calculting the gradients at every batch, not at every data point. Since it is an expensive gradient calculation, so  we reduce the number of times we do it by increasing the batch size. This has the benefit of reducing overfitting by computing the gradient over a large sample of images.

In [8]:
trainloader = torch.utils.data.DataLoader(trainset, batch_size=512, shuffle=True, num_workers=2)
validloader = torch.utils.data.DataLoader(validset, batch_size=512, shuffle=True, num_workers=2)
criterion = nn.CrossEntropyLoss()

In [9]:
def validation(net):
    valid_loss = 0
    with torch.no_grad():
        for data in validloader:
            images, labels = data
            outputs = net(images)
            loss = criterion(outputs, labels)
            valid_loss += loss.item()
    return valid_loss

In [10]:
n_epoch = 20

def train(net):
    optimizer = torch.optim.SGD(net.parameters(), lr=0.1, momentum=0.9)
    train_history = []
    valid_history = []
    for epoch in range(n_epoch):
        train_loss = 0.0
        for i, data in enumerate(trainloader, 0):
            # to complete
            optimizer.step()
            train_loss += loss.item()
        valid_loss = validation(net)
        train_history.append(train_loss)
        valid_history.append(valid_loss)
        print('Epoch %02d: train loss %0.5f, validation loss %0.5f' % (epoch, train_loss, valid_loss))
    return train_history, valid_history

In [45]:
# %load solutions/ex2_2.py

We can have a look at the time necessary to train the network over one epoch.

In [46]:
net = SimpleNet()
n_epoch = 1
%time train_loss = train(net)
print(train_loss)

We write a function to valid the predictions.

In [12]:
def get_valid_predictions(net):
    validloader = torch.utils.data.DataLoader(validset, batch_size=4, shuffle=False)
    all_labels = np.array([])
    predictions = np.array([])
    with torch.no_grad():
        for data in validloader:
            images, labels = data
            outputs = net(images)
            _, predicted = torch.max(outputs.data, 1)
            all_labels = np.append(all_labels, labels.numpy())
            predictions = np.append(predictions, predicted.numpy())
    return all_labels, predictions

In [50]:
from sklearn.metrics import accuracy_score, classification_report

print('Accuracy: ', accuracy_score(predictions, y_valid))
print(classification_report(predictions, y_valid, target_names=labels_text))

<div class="alert alert-warning">
    
**Exercice** <br>
Using the `torch.nn.Conv2d` and `torch.nn.MaxPool2d` layer functions (or `torch.nn.functional.max_pool2d`), define and test the following network:
    
- Input layer: images of size $28\times 28$ with a single channel
- Convolutional layer of 32 feature maps with $3\times 3$ filters
- Max pooling layer by blocks of size $2 \times 2$
- Convolutional layer of 64 feature maps with $3\times 3$ filters
- Max pooling layer by blocks of size $2 \times 2$
- Fully connected layer with 128 ReLU neurons
- Fully connected output layer with 10 ReLU neurons
</div>

In [13]:
import torch.nn.functional as F

In [47]:
# %load solutions/ex3.py

In [48]:
net = ConvNet()
n_epoch = 1
%time train_loss = train(net)
print(train_loss)

We create a function to plot the both training and validation results, in order to see overfitting.

In [16]:
def plot_train_val(train, valid):
    fig, ax1 = plt.subplots()
    color = 'tab:red'
    ax1.set_ylabel('Training', color=color)
    ax1.plot(train, color=color)
    ax2 = ax1.twinx()
    color = 'tab:blue'
    ax2.set_ylabel('Validation', color=color)
    ax2.plot(valid, color=color)
    fig.tight_layout()

<div class="alert alert-warning">
    
**Exercice** <br>
Write a Dropout network for this dataset.
</div>

In [49]:
# %load solutions/ex4.py

In [50]:
net = DropoutNet()
n_epoch = 1
%time train_loss = train(net)
print(train_loss)

Now that we have these different networks, and that we have a function to train them, we want to compare their tendancy to overtfit the dataset. 

**Training the SimpleNet**

In [51]:
trainloader = torch.utils.data.DataLoader(trainset, batch_size=512, shuffle=True, num_workers=2)
validloader = torch.utils.data.DataLoader(validset, batch_size=512, shuffle=True, num_workers=2)
criterion = nn.CrossEntropyLoss()

n_epoch = 30
net = SimpleNet()
train_history_simplenet, valid_history_simplenet = train(net)
plot_train_val(train_history_simplenet, valid_history_simplenet)

**Training the ConvNet**

In [53]:
trainloader = torch.utils.data.DataLoader(trainset, batch_size=512, shuffle=True, num_workers=2)
validloader = torch.utils.data.DataLoader(validset, batch_size=512, shuffle=True, num_workers=2)
criterion = nn.CrossEntropyLoss()

n_epoch = 30
net = ConvNet()
train_history_convnet, valid_history_convnet = train(net)
plot_train_val(train_history_convnet, valid_history_convnet)

**Training the Dropout net**

In [54]:
trainloader = torch.utils.data.DataLoader(trainset, batch_size=512, shuffle=True, num_workers=2)
validloader = torch.utils.data.DataLoader(validset, batch_size=512, shuffle=True, num_workers=2)
criterion = nn.CrossEntropyLoss()

n_epoch = 30
net = DropoutNet()
train_history_dropnet, valid_history_dropnet = train(net)
plot_train_val(train_history_dropnet, valid_history_dropnet)

This is a first approach of overfitting with and without Dropout. You can vary the parameters of the networks.

# <a id="sec4"></a>4. Comparing Dropout to No-Drop : the MNIST example

The MNIST handwritten digit classification task (LeCun et al., 1998) consists of 28×28 black and white images, each containing a digit 0 to 9 (10-classes). Each digit in the 60 000 training images and 10 000 test images is normalized to fit in a 20×20 pixel box while preserving their aspect ratio.

Also, it is better to run this part of the notebook with a GPU.

<div class="alert alert-warning">
    
**Exercice** <br>
If you feel like doing it, try to write, as we did before, a No-Drop Network and a DropOut Network, in order to study overfitting with the MNIST.
    
You could use Conv2d with maxpool and Linear in a NoDrop, and Dropout2d to drop out...
</div>

<div class="alert alert-warning">
    
**Exercice** <br>
If you don't feel like writing it entirely, you can study the code of Nikolaos Agadakos, and use it to return the results you need to study overfitting the MNIST.
</div>

**No-Drop Study**

In [38]:
import torch
import torchvision
import torchvision.datasets as tdata
import torchvision.transforms as tTrans
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optm
import matplotlib.pyplot as plt

# Python Imaging Library
import PIL
import numpy as np
import sys as sys

device  = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [55]:
trainAcc = 0
trainLoss= 1
testAcc  = 2
testLoss = 3

batch   = 64
epochs  = 25
gamma   = 0.01
momnt   = 0.5

mean = 0.1307
std  = 0.3081
comArgs = {'shuffle': True,'num_workers': 1, 'pin_memory': True} if torch.cuda.is_available() else {}

In [42]:
transform = tTrans.Compose([tTrans.ToTensor(), tTrans.Normalize( (mean,), (std,) )])

mnistTrainset = tdata.MNIST(root='./data_MNIST', train=True, download=True, transform=transform)
mnistTestset = tdata.MNIST(root='./data_MNIST', train=False, download=True, transform=transform)
transform = tTrans.Compose([tTrans.ToTensor(), tTrans.Normalize( (mean,), (std,) )])


trainLoader = torch.utils.data.DataLoader(mnistTrainset, batch_size = batch, **comArgs )
testLoader = torch.utils.data.DataLoader(mnistTestset, batch_size = 10*batch, **comArgs)

In [57]:
class Net(nn.Module):

    # Class variables for measures.
    trainAcc = 0
    trainLoss= 0
    testAcc  = 0
    testLoss = 0

    history = [[],[],[],[]]

    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3)
        self.fc1 = nn.Linear(64*144, 128)
        self.fc2 = nn.Linear(128, 10)


    def forward(self, x):
        x = F.relu(self.conv1(x), 2)
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, 64*144)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        x = F.log_softmax(x, dim=1)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  
        num_features = 1
        for s in size:       
            num_features *= s
        return num_features
  

    def train(self, args, device, indata, optim):
        true = 0
        acc  = 0
        for idx, (img, label) in enumerate(indata):
            data, label = img.to(device), label.to(device)
            
            output = self.forward(data)
            pred   = output.max(dim = 1, keepdim = True)
            true  += label.eq(pred[1].view_as(label)).sum().item()
            
            loss = F.cross_entropy(output, label)
            
            optim.zero_grad()
            loss.backward()            
            optim.step()

        acc = true/len(trainLoader.dataset)
        self.history[trainLoss].append(loss)   
        self.history[trainAcc].append(acc)


    def test(self, device, testLoader):
        print("Commence Testing!")        
        loss = 0 
        true = 0
        acc  = 0
       
        with torch.no_grad():
            for data, label in testLoader:
                data, label = data.to(device), label.to(device)
                
                output = self.forward(data)
        
                loss  += F.cross_entropy(output, label, reduction = 'elementwise_mean').item()

                pred   = output.max(dim = 1, keepdim = True)
                
                true  += label.eq(pred[1].view_as(label)).sum().item()
                
        acc = true/len(testLoader.dataset)
        self.history[testAcc].append(acc)
        self.history[testLoss].append(loss/len(testLoader.dataset)) 
        # Print accuracy report!
        print("Accuracy: {} ({} / {})".format(acc, true,
                                              len(testLoader.dataset)))

    def report(self):

        print("Current stats of MNIST_NET:")
        print("Accuracy:      {}" .format(self.trainAcc))
        print("Training Loss: {}" .format(self.trainLoss))
        print("Test Accuracy: {}" .format(self.testAcc))
        print("Test Loss:     {}" .format(self.testLoss))


# Execution

def main():
    print("######### Initiating MNIST N0-DROP Network Training #########\n")

    model = Net().to(device)
    optim = optm.SGD(model.parameters(), lr=gamma, momentum=momnt)

    for e in range(epochs):
        print("Epoch: {} start ------------\n".format(e))
        args = e
        model.train(args, device, trainLoader, optim)
        model.test(device, testLoader)
    model.report()

# Define behavior if this module is the main executable.
if __name__ == '__main__':
    main()

**DropOut Study**

In [ ]:
import torch
import torchvision
import torchvision.datasets as tdata
import torchvision.transforms as tTrans
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optm
import matplotlib.pyplot as plt

# Python Imaging Library
import PIL
import numpy as np
import sys as sys


# Parameters
batch   = 64
epochs  = 20
gamma   = 0.01
momnt   = 0.5
device  = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

comArgs = {'shuffle': True,'num_workers': 1, 'pin_memory': True} if torch.cuda.is_available() else {}


mean = 0.1307
std  = 0.3081

transform = tTrans.Compose([tTrans.ToTensor(), tTrans.Normalize( (mean,), (std,) )])
mnistTrainset = tdata.MNIST(root='./data', train=True, download=True, transform=transform)
mnistTestset = tdata.MNIST(root='./data', train=False, download=True, transform=transform)


trainLoader = torch.utils.data.DataLoader(mnistTrainset, batch_size = batch, **comArgs )
testLoader = torch.utils.data.DataLoader(mnistTestset, batch_size = 10*batch, **comArgs)


class Net(nn.Module):

    accuracy = 0
    trainLoss= 0
    testLoss = 0

    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)

        self.drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)


    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.drop(self.conv2(x)), 2))

        # Flatten in to 1D to feed to dense Layer.
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        x = F.log_softmax(x, dim=1)
        return x

    def forward_no_drop(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2(x), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        x = F.log_softmax(x, dim=1)
        return x

    def train(self, args, device, indata, optim):

        for idx, (img, label) in enumerate(indata):
            data, label = img.to(device), label.to(device)
            # forward pass calculate output of model
            output      = self.forward(data)
            # compute loss
            loss        = F.nll_loss(output, label)


            optim.zero_grad()
            loss.backward()            
            optim.step()

          
        self.trainLoss = loss   


    def test(self, device, testLoader):
        print("In Testing Function!")        
        loss = 0 
        true = 0
        acc  = 0
 
        with torch.no_grad():
            for data, label in testLoader:
                data, label = data.to(device), label.to(device)
   
                output = self.forward(data)
     
                loss  += F.nll_loss(output, label, reduction = 'sum').item()

                pred   = output.max(dim = 1, keepdim = True)
                true  += label.eq(pred[1].view_as(label)).sum().item()
        acc = true/len(testLoader.dataset)
        self.accuracy = acc
        self.testLoss = loss 

        print("Accuracy: {} ({} / {})".format(acc, true,
                                              len(testLoader.dataset)))

    def report(self):

        print("Current stats of MNIST_NET:")
        print("Accuracy:      {}" .format(self.accuracy))
        print("Training Loss: {}" .format(self.trainLoss))
        print("Test Loss:     {}" .format(self.testLoss))


# Execution
#-----------------------------
def main():
    print("######### Initiating MNIST network training #########\n")

    model = Net().to(device)
    optim = optm.SGD(model.parameters(), lr=gamma, momentum=momnt)

    for e in range(epochs):
        print("Epoch: {} start ------------\n".format(e))
        args = e
        model.train(args, device, trainLoader, optim)
        model.test(device, testLoader)

    # Final report
    model.report()
# Define behavior if this module is the main executable.
if __name__ == '__main__':
    main()

# 5. <a id="sec5"></a> A glance at DropConnect theory



**The idea:**
DropConnect is the generalization of Dropout in which each connection, rather than each output unit, can be dropped with probability $1 − p$. DropConnect is similar to Dropout as it introduces dynamic sparsity within the model, but differs in that the sparsity is on the weights W, rather than the output vectors of a layer. In other words, the fully connected layer with DropConnect becomes a sparsely connected layer in which the connections are chosen at random during the training stage. Note that this is not equivalent to setting W to be a fixed sparse matrix during training.

For a DropConnect layer, the output is given as:

$$r=a((M*W)\upsilon)$$

where :
- $r = [r_1,r_2,r_3,\dots,r_d]^T$ output of the layer
- $M$ is a binary matrix encoding the connection information and $M_{ij}$ ∼ Bernoulli(p)
- $*$ denotes [element wise product](https://en.wikipedia.org/wiki/Hadamard_product_(matrices)#:~:text=In%20mathematics%2C%20the%20Hadamard%20product,of%20the%20original%20two%20matrices.).
- $W$ weight parameters (of size $d$ × $n$)
- $\upsilon = [\upsilon_1,\upsilon_2,\upsilon_3,\dots,\upsilon_n]^T$ input of the layer 

Each element of the mask M is drawn independently for each example during training, essentially instantiating a different connectivity for each example seen. Additionally, the biases are also masked out during training.

**Visual comparision between DropOut et DropConnect**

**DropOut**<br>
<img src="img/dropout_network_2.png"><br>

**DropConnect**<br>
<img src="img/dropconnect_network.png">


# 6. <a id="sec6"></a> DropConnect propositions of implementation



## 6.1 Implementations elements

In their paper [Regularization of Neural Network using DropConnect](http://proceedings.mlr.press/v28/wan13.pdf), the autors do not give any piece of code.

But, because they said that DropConnect had better results than DropOut and No-Drop networks, data scientists tried to implement it, and to use it.

But, it is now known in the community that DropConnect is hard to implement, and that its results are not suffiscient, and are not worth it.

One of the implementation found is the following one. A proposition of implementation for a DropConnect layer could be the following one. The idea is to create a Python class able to drop the weights for a given module. First, we want to create a function `_weight_drop` with the help for `torch.nn.functional.dropout` which is described [here](https://pytorch.org/docs/stable/nn.functional.html#dropout-functions).


You can find this implementation [here](https://pytorchnlp.readthedocs.io/en/latest/_modules/torchnlp/nn/weight_drop.html).

<div class="alert alert-warning">
    
**Exercice** <br>
Use torch to complete the given _weight_drop function.
</div>

In [20]:
from torch.nn import Parameter

In [21]:
def _weight_drop(module, weights, dropout):
    """
    Helper for `WeightDrop`.
    """

    for name_w in weights:
        w = getattr(module, name_w)
        del module._parameters[name_w]
        module.register_parameter(name_w + '_raw', Parameter(w))

    original_module_forward = module.forward

    def forward(*args, **kwargs):
        for name_w in weights:
            raw_w = getattr(module, name_w + '_raw')
            # to complete...
            setattr(module, name_w, w)

        return original_module_forward(*args, **kwargs)

    setattr(module, 'forward', forward)


In [26]:
# %load solutions/ex4.py

In [25]:
class WeightDrop(torch.nn.Module):
    """
    The weight-dropped module applies recurrent regularization through a DropConnect mask on the
    hidden-to-hidden recurrent weights.

    **Thank you** to Sales Force for their initial implementation of :class:`WeightDrop`. Here is
    their `License
    <https://github.com/salesforce/awd-lstm-lm/blob/master/LICENSE>`__.

    Args:
        module (:class:`torch.nn.Module`): Containing module.
        weights (:class:`list` of :class:`str`): Names of the module weight parameters to apply a
          dropout too.
        dropout (float): The probability a weight will be dropped.

    Example:

        >>> from torchnlp.nn import WeightDrop
        >>> import torch
        >>>
        >>> torch.manual_seed(123)
        <torch._C.Generator object ...
        >>>
        >>> gru = torch.nn.GRUCell(2, 2)
        >>> weights = ['weight_hh']
        >>> weight_drop_gru = WeightDrop(gru, weights, dropout=0.9)
        >>>
        >>> input_ = torch.randn(3, 2)
        >>> hidden_state = torch.randn(3, 2)
        >>> weight_drop_gru(input_, hidden_state)
        tensor(... grad_fn=<AddBackward0>)
    """

    def __init__(self, module, weights, dropout=0.0):
        super(WeightDrop, self).__init__()
        _weight_drop(module, weights, dropout)
        self.forward = module.forward



class WeightDropLSTM(torch.nn.LSTM):
    """
    Wrapper around :class:`torch.nn.LSTM` that adds ``weight_dropout`` named argument.

    Args:
        weight_dropout (float): The probability a weight will be dropped.
    """

    def __init__(self, *args, weight_dropout=0.0, **kwargs):
        super().__init__(*args, **kwargs)
        weights = ['weight_hh_l' + str(i) for i in range(self.num_layers)]
        _weight_drop(self, weights, weight_dropout)



class WeightDropGRU(torch.nn.GRU):
    """
    Wrapper around :class:`torch.nn.GRU` that adds ``weight_dropout`` named argument.

    Args:
        weight_dropout (float): The probability a weight will be dropped.
    """

    def __init__(self, *args, weight_dropout=0.0, **kwargs):
        super().__init__(*args, **kwargs)
        weights = ['weight_hh_l' + str(i) for i in range(self.num_layers)]
        _weight_drop(self, weights, weight_dropout)



class WeightDropLinear(torch.nn.Linear):
    """
    Wrapper around :class:`torch.nn.Linear` that adds ``weight_dropout`` named argument.

    Args:
        weight_dropout (float): The probability a weight will be dropped.
    """

    def __init__(self, *args, weight_dropout=0.0, **kwargs):
        super().__init__(*args, **kwargs)
        weights = ['weight']
        _weight_drop(self, weights, weight_dropout)

But some pointed out the fact that this implementation was quite difficult, and that it was possible to use the following piece of code. You can find the discussion over the implementation [here](https://discuss.pytorch.org/t/dropconnect-implementation/70921).

In [23]:
def train_drop(model):

    for i in range(num_batches):

        orig_params = []
        for n, p in model.named_parameters():
            orig_params.append(p.clone())
            p.copy_(F.dropout(p.data, p=drop_prob) * (1 - drop_prob))

        output = model(input)
        loss = nn.CrossEntropyLoss()(output, label)
        optimizer.zero_grad()
        loss.backward()

        for orig_p, (n, p) in zip(orig_params, model.named_parameters()):  
            p.copy_(orig_p)

        optimizer.step()

## 6.2 Model using DropConnect

In their paper [Regularization of Neural Network using DropConnect](http://proceedings.mlr.press/v28/wan13.pdf), the model used by the authors is the following one.





We consider a standard model architecture composed of :

1. A feature extractor: $\upsilon = g(x; W_g)$ where $\upsilon$ are the output features, $x$ is input data to the overall model, and $W_g$ are parameters for the feature extractor. We choose $g()$ to be a multi-layered convolutional neural
network (CNN), with $W_g$ being the convolutional filters (and biases) of the CNN.
2. A DropConnect layer: $r = a(u) = a((M * W)\upsilon)$ where $\upsilon$ is the output of the feature extractor, $W$ is a fully connected weight matrix, $a$ is a non-linear activation function and $M$ is the binary mask matrix.
3. A Softmax Classification Layer: $o = s(r; W_s)$ takes as input $r$ and uses parameters $W_s$ to map this to a $k$ dimensional output ($k$ being the number of classes).
4. Cross Entropy Loss: $A(y, o) = - \sum_{i=1}^k y_i log(o_i)$ takes probabilities $o$ and the ground truth labels $y$ as input.



<img src="img/model_anatomy.png">

**3. Softmax layer**

To create a Softmax layer with Pytorch, you can use `torch.nn.Softmax(dim: Optional[int] = None)`. The documentation can be found [here](https://pytorch.org/docs/stable/generated/torch.nn.Softmax.html).

Training has the following steps :

- selecting an example $x$ from the training set and extracting features for that example, $\upsilon$. These features are input to the DropConnect layer where a mask matrix $M$ is first drawn from a Bernoulli($p$) distribution to mask out elements of both the weight matrix and the biases in the DropConnect layer. (A key component to successfully training with DropConnect is the selection of a different mask for each training example. Selecting a single mask for a subset of training examples does not regularize the model enough in practice.)

- apply a chosen mask to the weights and biases in order to compute the input to the activation function. This results in $r$, the input to the softmax layer which outputs class predictions from which cross entropy between the ground truth labels is computed. The parameters throughout the model $\theta$ then can be updated via stochastic gradient descent (SGD) by backpropagating gradients of the loss function with respect to the parameters, $A'_\theta$.

To update the weight matrix $W$ in a DropConnect layer, the mask is applied to the gradient to update only those elements
that were active in the forward pass. Additionally, when passing gradients down to the feature extractor, the masked weight matrix $M*W$ is used.

<img src="img/algo_1.png">

As an example, the results obtained by the authors for the MNIST are the following ones. 

<img src="img/results_MNIST_1.png">

<img src="img/results_MNIST.png">

As the model size increases, NoDrop overfits while both Dropout and DropConnect improve performance. DropConnect consistently gives a lower error rate than Dropout.

Feel free to try to implement a DropConnect layer to use it with the MNIST, as an opening... 